# Informations attendues potentiel solaire - Checklist

# Sommaire
* [SOURCES](#SOURCES)
    * [ANNUAIRE](#ANNUAIRE)
    * [BDD TOPO](#BDD-TOPO)
    * [POTENTIEL GISEMENT SOLAIRE BRUT AU BATI](#POTENTIEL-GISEMENT-SOLAIRE-BRUT-AU-BATI)
* [CALCUL DE PROXIMITE](#CALCUL-DE-PROXIMITE) 

références: 
- https://outline.services.dataforgood.fr/doc/contraintes-a-respecter-pour-linstallation-de-panneaux-solaires-W2W2IaNwKn
- https://data-iau-idf.opendata.arcgis.com/datasets/iau-idf::le-potentiel-solaire-des-toitures/about
- https://outline.services.dataforgood.fr/doc/ressources-algorithme-9afGJyytC8
- https://geocatalogue.apur.org/catalogue/srv/fre/catalog.search#/metadata/urn:apur:potentiel_gisement_solaire_brut_au_bati

## Contraintes à respecter
- Logistique
    - Pente du toit
    - Orientation au soleil
    - Latitude
    - Indice de rayonnement solaire local
    - Bâtiments classés
    - Bâtiments en zone non classée
    - Ombre (Bâtiments adjacents & végétations)
    - Eléments présents sur le toit
    - Structure porteuse (est-ce que le bâtiment peut supporter le poids?)
- Réseaux

# Potentiel Solaire - Attributs importants
- surf_util : Surface présentant une irradiation suffisante (> 900 kWh/m².an) et sans obstacles (comme des cheminées, velux, aérations, etc.). Cette surface est réellement utilisable pour l'installation de panneaux solaires.
- indic2 : Valeur numérique correspondant au gisement solaire. Les valeurs possibles sont :
1 : Non favorable
1 : Faible
2 : Intermédiaire
3 : Important
- gisement : Classification du gisement solaire en termes qualitatifs : "important", "intermédiaire", "faible", ou "Non favorable".
- moyenne2 : Potentiel solaire moyen au m² du bâtiment, exprimé en kWh/m².an. Cet indicateur représente la quantité d'énergie solaire reçue sur la surface utile.
- production : La production annuelle théorique en kWh/an, qui est une estimation de la quantité d'énergie solaire que la toiture pourrait produire sur une année.
- forme : Forme de la toiture estimée à partir d'un modèle numérique de surface. La forme de la toiture peut influencer l'ensoleillement et donc le potentiel solaire.
- eq_pano : La surface utile en équivalent panneaux solaires. Cet attribut indique le nombre de panneaux solaires susceptibles d'être installés sur la toiture.
- eq_surf : L'équivalent en superficie de l'équivalent panneau solaire, qui donne une idée de la surface disponible pour l'installation de panneaux solaires.

# Qualité du rattachement de l'annuaire avec les surfaces / bâtiments de la BD TOPO
"un des points importants à regarder en l'état c'est la qualité du rattachement de l'annuaire avec les surfaces / bâtiments de la BD TOPO. Pour le moment c'est une règle sur la proximité géographique qui est utilisée mais il y a d'autres métadonnées qui peuvent nous éclairer je pense. Par exemple :
le champs toponyme du layer zone d'activité de la BD TOPO (ex : "Ecole Elementaire Victor Hugo") avec le champs nom_etablissement de l'annuaire (ex: "Ecole élémentaire Victor Hugo")
le champs nature du layer zone d'activité de la BD TOPO (9 valeurs possibles) avec type_etablissement de l'annuaire (8 valeurs possibles)
En creusant, c'est clairement pas l'égalité à chercher mais peut etre calculer la proximité des valeurs (ou juste les comparer) pour voir si on s'est pas complètement loupé. Ou trouver une piste pour les établissements de l'annuaire sans surface d'activité.
Sachant que j'ai pas regardé tous les champs et il y a peut être d'autres informations à utiliser."

In [2]:
import geopandas as gpd
from pathlib import Path
import matplotlib.pyplot as plt
import contextily as cx
import fiona
import pandas as pd

# Potentiel solaire package
from potentiel_solaire.constants import DATA_FOLDER

In [11]:
!extract-sample-data

2025-02-12 18:33:21,693 - INFO - root - /Users/hui-guan/Documents/Developpement/data4good/13_potentiel_solaire/algorithme/potentiel_solaire/extract_sample_data.py - main - GPKG file potentiel-solaire.gpkg already exists, skipping conversion.
2025-02-12 18:33:21,694 - INFO - root - /Users/hui-guan/Documents/Developpement/data4good/13_potentiel_solaire/algorithme/potentiel_solaire/extract_sample_data.py - main - GPKG file potentiel-gisement-solaire-brut-au-bati.gpkg already exists, skipping conversion.
2025-02-12 18:33:21,694 - INFO - root - /Users/hui-guan/Documents/Developpement/data4good/13_potentiel_solaire/algorithme/potentiel_solaire/extract_sample_data.py - main - Folder for BDTOPO_3-4_TOUSTHEMES_GPKG_LAMB93_D093_2024-12-15.7z already exists, skipping download and extraction.
2025-02-12 18:33:21,694 - INFO - root - /Users/hui-guan/Documents/Developpement/data4good/13_potentiel_solaire/algorithme/potentiel_solaire/extract_sample_data.py - main - Folder for PARCELLAIRE-EXPRESS_1-1__

# SOURCES

## ANNUAIRE

In [44]:
annuaire = gpd.read_file(DATA_FOLDER / 'fr-en-annuaire-education.geojson')
annuaire = annuaire.to_crs(4326)
#annuaire.shape
annuaire[annuaire.code_commune=='93066'].sample(15) # Saint-Denis

,identifiant_de_l_etablissement,nom_etablissement,type_etablissement,statut_public_prive,adresse_1,adresse_2,adresse_3,code_postal,code_commune,nom_commune,code_departement,code_academie,code_region,ecole_maternelle,ecole_elementaire,voie_generale,voie_technologique,voie_professionnelle,telephone,fax,web,mail,restauration,hebergement,ulis,apprentissage,segpa,section_arts,section_cinema,section_theatre,section_sport,section_internationale,section_europeenne,lycee_agricole,lycee_militaire,lycee_des_metiers,post_bac,appartenance_education_prioritaire,greta,siren_siret,nombre_d_eleves,fiche_onisep,position,type_contrat_prive,libelle_departement,libelle_academie,libelle_region,coordx_origine,coordy_origine,epsg_origine,nom_circonscription,latitude,longitude,precision_localisation,date_ouverture,date_maj_ligne,etat,ministere_tutelle,multi_uai,rpi_concentre,rpi_disperse,code_nature,libelle_nature,code_type_contrat_prive,pial,etablissement_mere,type_rattachement_etablissement_mere,code_circonscription,code_zone_animation_pedagogique,libelle_zone_animation_pedagogique,code_bassin_formation,libelle_bassin_formation,geometry
57497,0930223M,Ecole élémentaire Louis Blériot,Ecole,Public,5 passage des Ecoles,None,93200 ST DENIS,93200,93066,Saint-Denis,093,24,11,0.0,1.0,None,None,None,0183722140,None,None,ce.0930223M@ac-creteil.fr,1.0,0.0,0.0,None,None,None,None,None,None,None,None,None,None,None,None,REP+,None,21930066200174,184.0,None,None,SANS OBJET,Seine-Saint-Denis,Créteil,Ile-de-France,654121.2,6870994.5,EPSG:2154,Circonscription d'inspection du 1er degré de S...,48.937287,2.373748,Numéro de rue,1965-05-01,2025-02-02,OUVERT,MINISTERE DE L'EDUCATION NATIONALE,0,0.0,None,151,ECOLE DE NIVEAU ELEMENTAIRE,99,None,None,None,0932058G,None,None,None,None,POINT (2.37375 48.93729)
41467,0932612J,Ecole maternelle La Confluence,Ecole,Public,55 rue Charles Michels,None,93200 ST DENIS,93200,93066,Saint-Denis,093,24,11,1.0,0.0,None,None,None,0183722299,None,None,ce.0932612J@ac-creteil.fr,1.0,0.0,0.0,None,None,None,None,None,None,None,None,None,None,None,None,REP,None,21930066201230,114.0,None,None,SANS OBJET,Seine-Saint-Denis,Créteil,Ile-de-France,651859.7,6870507.2,EPSG:2154,Circonscription d'inspection du 1er degré de S...,48.932739,2.342935,Numéro de rue,2015-09-01,2025-02-02,OUVERT,MINISTERE DE L'EDUCATION NATIONALE,0,0.0,None,101,ECOLE MATERNELLE,99,0932613K,None,None,0931036W,None,None,None,None,POINT (2.34294 48.93274)
57479,0930166A,Ecole élémentaire Pierre Sémard,Ecole,Public,2 rue Eugène Henaff,None,93200 ST DENIS,93200,93066,Saint-Denis,093,24,11,0.0,1.0,None,None,None,0183722150,None,None,ce.0930166A@ac-creteil.fr,1.0,0.0,0.0,None,None,None,None,None,None,None,None,None,None,None,None,REP,None,21930066200257,252.0,None,None,SANS OBJET,Seine-Saint-Denis,Créteil,Ile-de-France,652397.8,6872123.9,EPSG:2154,Circonscription d'inspection du 1er degré de S...,48.947317,2.350098,Numéro de rue,1965-05-01,2025-02-02,OUVERT,MINISTERE DE L'EDUCATION NATIONALE,0,0.0,None,151,ECOLE DE NIVEAU ELEMENTAIRE,99,None,None,None,0932609F,None,None,None,None,POINT (2.3501 48.94732)
59685,0931232J,Collège Henri Barbusse,Collège,Public,23 rue de Sevran,None,None,93200,93066,Saint-Denis,093,24,11,NaN,NaN,0,0,0,01 49 40 10 35,01 42 35 23 49,https://barbusse-saintdenis.webcollege.seinesa...,ce.0931232j@ac-creteil.fr,1.0,0.0,1.0,0,1,0,0,0,0,0,0,0,0,0,0,REP,0,19931232300017,533.0,https://www.onisep.fr/http/redirection/etablis...,None,SANS OBJET,Seine-Saint-Denis,Créteil,Ile-de-France,654352.2,6871315.3,EPSG:2154,None,48.940188,2.376867,Numéro de rue,1970-03-25,2025-02-02,OUVERT,MINISTERE DE L'EDUCATION NATIONALE,0,0.0,None,340,COLLEGE,99,None,None,None,None,None,None,24931,93 BASSIN 1 (DIST. 1 ET 2),POINT (2.37687 48.94019)
68449,0931388D,Section d'enseignement professionnel du Lycée ...,Lycée,Public,Place du 8 Mai 1945,BP 85,93203 ST DENIS CEDEX,93203,93066,Saint-Denis,093,24,11,NaN,NaN,None,None,None,0149718700,None,None,ce.0931388D@ac-creteil.fr,0.0,0.0,0.0,None,0,No

In [43]:
# Explo
pd.set_option('display.max_columns', 100)
annuaire[(annuaire.nom_etablissement=='Ecole élémentaire Daniel Sorano') & (annuaire.code_commune=='93066')]

,identifiant_de_l_etablissement,nom_etablissement,type_etablissement,statut_public_prive,adresse_1,adresse_2,adresse_3,code_postal,code_commune,nom_commune,code_departement,code_academie,code_region,ecole_maternelle,ecole_elementaire,voie_generale,voie_technologique,voie_professionnelle,telephone,fax,web,mail,restauration,hebergement,ulis,apprentissage,segpa,section_arts,section_cinema,section_theatre,section_sport,section_internationale,section_europeenne,lycee_agricole,lycee_militaire,lycee_des_metiers,post_bac,appartenance_education_prioritaire,greta,siren_siret,nombre_d_eleves,fiche_onisep,position,type_contrat_prive,libelle_departement,libelle_academie,libelle_region,coordx_origine,coordy_origine,epsg_origine,nom_circonscription,latitude,longitude,precision_localisation,date_ouverture,date_maj_ligne,etat,ministere_tutelle,multi_uai,rpi_concentre,rpi_disperse,code_nature,libelle_nature,code_type_contrat_prive,pial,etablissement_mere,type_rattachement_etablissement_mere,code_circonscription,code_zone_animation_pedagogique,libelle_zone_animation_pedagogique,code_bassin_formation,libelle_bassin_formation,geometry
41191,0930306C,Ecole élémentaire Daniel Sorano,Ecole,Public,3 boulevard Félix Faure,None,93200 ST DENIS,93200,93066,Saint-Denis,093,24,11,0.0,1.0,None,None,None,0183722220,None,None,ce.0930306C@ac-creteil.fr,1.0,0.0,0.0,None,None,None,None,None,None,None,None,None,None,None,None,REP,None,21930066200208,292.0,None,None,SANS OBJET,Seine-Saint-Denis,Créteil,Ile-de-France,652972.5,6871169.1,EPSG:2154,Circonscription d'inspection du 1er degré de S...,48.938774,2.35805,Numéro de rue,1965-05-01,2025-02-02,OUVERT,MINISTERE DE L'EDUCATION NATIONALE,0,0.0,None,151,ECOLE DE NIVEAU ELEMENTAIRE,99,None,None,None,0932609F,None,None,None,None,POINT (2.35805 48.93877)


## BDD TOPO

In [16]:
GPKG = DATA_FOLDER / "BDTOPO_3-4_TOUSTHEMES_GPKG_LAMB93_D093_2024-12-15/BDTOPO/1_DONNEES_LIVRAISON_2024-12-00134/BDT_3-4_GPKG_LAMB93_D093-ED2024-12-15/BDT_3-4_GPKG_LAMB93_D093-ED2024-12-15.gpkg"
communes = gpd.read_file(GPKG, layer="commune")

# on filtre sur saint-denis
saint_denis = communes[communes.code_insee == "93066"].to_crs(4326)


In [15]:
# Exploration des layers
layers = fiona.listlayers(GPKG)
layers

['troncon_de_route',
 'route_numerotee_ou_nommee',
 'itineraire_autre',
 'troncon_de_voie_ferree',
 'equipement_de_transport',
 'piste_d_aerodrome',
 'aerodrome',
 'point_de_repere',
 'non_communication',
 'point_du_reseau',
 'voie_ferree_nommee',
 'toponymie_transport',
 'batiment',
 'cimetiere',
 'construction_lineaire',
 'construction_ponctuelle',
 'construction_surfacique',
 'reservoir',
 'ligne_orographique',
 'pylone',
 'terrain_de_sport',
 'toponymie_bati',
 'cours_d_eau',
 'troncon_hydrographique',
 'bassin_versant_topographique',
 'plan_d_eau',
 'surface_hydrographique',
 'noeud_hydrographique',
 'detail_hydrographique',
 'toponymie_hydrographie',
 'zone_d_habitation',
 'lieu_dit_non_habite',
 'detail_orographique',
 'toponymie_lieux_nommes',
 'canalisation',
 'ligne_electrique',
 'poste_de_transformation',
 'erp',
 'zone_d_activite_ou_d_interet',
 'toponymie_services_et_activites',
 'voie_nommee',
 'parc_ou_reserve',
 'foret_publique',
 'toponymie_zones_reglementees',
 'haie'

In [37]:
zone_d_activite=gpd.read_file(GPKG, layer="zone_d_activite_ou_d_interet")
zone_d_activite[zone_d_activite.toponyme.str.contains("Daniel Sorano")==True]

,cleabs,categorie,nature,nature_detaillee,toponyme,statut_du_toponyme,importance,fictif,etat_de_l_objet,date_creation,date_modification,date_d_apparition,date_de_confirmation,sources,identifiants_sources,methode_d_acquisition_planimetrique,precision_planimetrique,identifiant_voie_ban,nom_commercial,geometry
3805,SURFACTI0000000330501966,Sport,Complexe sportif couvert,Gymnase,Espace Daniel Sorano,Collecté,5,False,En service,2013-12-11 16:31:01.246,2024-05-17 09:24:53.802,2012/07/16,2024-04-25,MSS 2024,MSS:I940800015/MSS:I940800020,Calculé,20.0,None,None,"MULTIPOLYGON (((658611.9 6861211.2, 658615.2 6..."
5604,SURFACTI0000000241613991,Culture et loisirs,Espace public,Place,Place Daniel Sorano,Collecté,5,False,En service,2010-08-06 12:48:03.897,2022-12-10 09:25:25.655,None,NaT,None,None,Orthophotographie,3.0,77258_2370,None,"MULTIPOLYGON (((673338.4 6860395.6, 673329.8 6..."
9968,SURFACTI0000000002555607,Science et enseignement,Enseignement primaire,Groupe scolaire,École Élémentaire Daniel Sorano,Collecté,5,False,En service,2006-05-22 16:31:28.175,2024-04-17 14:57:25.903,None,2024-04-09,MEN 2024,MEN:0930306C,BDTopo,2.5,None,None,"MULTIPOLYGON (((653017.7 6871189.3, 653017.4 6..."


In [57]:
toponymie_services_et_activites=gpd.read_file(GPKG, layer="toponymie_services_et_activites")
toponymie_services_et_activites

,cleabs_de_l_objet,classe_de_l_objet,nature_de_l_objet,graphie_du_toponyme,source_du_toponyme,statut_du_toponyme,date_du_toponyme,langue_du_toponyme,geometry
0,SURFACTI0000002200044540,Zone d'activité ou d'intérêt,Espace public,parvis arnaud beltrame,Mairie information officielle,Collecté,2018-12-04,None,POINT (668569.148 6866977.15)
1,SURFACTI0000000001728352,Zone d'activité ou d'intérêt,Divers industriel,ancienne chocolaterie,Terrain,Validé,2000-09-19,None,POINT (672381.351 6861968.3)
2,SURFACTI0000002321375908,Zone d'activité ou d'intérêt,Salle de spectacle ou conférence,académie du climat,Mairie,Collecté,2022-09-28,None,POINT (652729.5 6861976.2)
3,SURFACTI0000002102009394,Zone d'activité ou d'intérêt,Espace public,square du pont maréchal juin,Mairie,Collecté,1900-01-01,None,POINT (646336.562 6866180.55)
4,SURFACTI0000002102009394,Zone d'activité ou d'intérêt,Espace public,square du pont bineau,Terrain;Mairie,Collecté,2018-12-05,None,POINT (646336.562 6866180.55)
...,...,...,...,...,...,...,...,...,...
12720,SURFACTI0000000002556160,Zone d'activité ou d'intérêt,Enseignement primaire,groupe scolaire varenne,Ministère,Collecté,2015-06-05,None,POINT (665833.205 6860873.85)
12721,SURFACTI0000000001728414,Zone d'activité ou d'intérêt,Zone industrielle,parc d'activités du pré fusé,IGN,Collecté,2013-10-18,None,POINT (670580.537 6853934.1)
12722,POSTRANS0000000001735419,Poste de transformation,None,poste électrique de villevaudé,IGN,Validé,1999-12-16,None,POINT (675254.443 6867226.5)
12723,POSTRANS0000000001735429,Poste de transformation,None,poste électrique du morbras,IGN,Validé,1994-07-05,None,POINT (674164.344 6853245.85)


In [73]:
toponymie_services_et_activites[toponymie_services_et_activites.cleabs_de_l_objet=="SURFACTI0000000002556160"]

,cleabs_de_l_objet,classe_de_l_objet,nature_de_l_objet,graphie_du_toponyme,source_du_toponyme,statut_du_toponyme,date_du_toponyme,langue_du_toponyme,geometry
12720,SURFACTI0000000002556160,Zone d'activité ou d'intérêt,Enseignement primaire,groupe scolaire varenne,Ministère,Collecté,2015-06-05,None,POINT (665833.205 6860873.85)


In [38]:
zone_d_activite[(zone_d_activite.toponyme.str.contains("Daniel Sorano")==True)&(zone_d_activite.methode_d_acquisition_planimetrique=="BDTopo")][["toponyme","nature","nature_detaillee","date_modification"]]


,toponyme,nature,nature_detaillee,date_modification
9968,École Élémentaire Daniel Sorano,Enseignement primaire,Groupe scolaire,2024-04-17 14:57:25.903


## POTENTIEL GISEMENT SOLAIRE BRUT AU BATI


En **gras** les attributs importants pour le projet

- N_SQ_EB: Identifiant informatique séquentiel
- **C_CAINSEE: Code INSEE de la commune/arrondissement**
- **C_ENS_MOY: Ensoleillement annuel moyen (KWh/m²/an)**
- M2_E_IN700: Surface d'ensoleillement inférieure à 700 KWh/m²/an
- M2_E_70_80: Surface d'ensoleillement comprise entre 700 et 800 KWh/m²/an
- M2_E_80_90: Surface d'ensoleillement comprise entre 800 et 900 KWh/m²/an
- **M2_E_90_10: Surface d'ensoleillement comprise entre 900 et 1000 KWh/m²/an**
- **M2_E_S1000: Surface d'ensoleillement supérieure à 1000 KWh/m²/an**
- **M2_E_TOT: Surface totale d'ensoleillement (m²)**
- SHAPE_LENGTH: Périmètre de l’emprise (m)
- SHAPE_AREA: Surface de l’emprise (m²)

In [11]:
potentiel_solaire = gpd.read_file(DATA_FOLDER / 'potentiel-gisement-solaire-brut-au-bati.geojson')
potentiel_solaire = potentiel_solaire.to_crs(4326)

In [14]:
potentiel_solaire[potentiel_solaire.c_cainsee==93066]

,geo_point_2d,geo_shape,objectid,n_sq_eb,n_sq_eb_or,c_cainsee,m2_e_in700,m2_e_70_80,m2_e_80_90,m2_e_90_10,m2_e_s1000,m2_e_tot,c_ens_moy,shape_leng,shape_area,geometry
33,"{ ""lon"": 2.3796635368542507, ""lat"": 48.9373916...",None,970114,930305028.0,930305028.0,93066.0,30,5,4,13,13,65,764.0,32.025768,63.961840,"POLYGON ((2.37974 48.93738, 2.37965 48.93734, ..."
207,"{ ""lon"": 2.3806389167648239, ""lat"": 48.9400496...",None,970618,930301319.0,930301319.0,93066.0,25,17,14,22,6,84,740.0,36.631041,83.173984,"POLYGON ((2.38072 48.94003, 2.38062 48.93999, ..."
362,"{ ""lon"": 2.3580500444470265, ""lat"": 48.9047674...",None,971039,930310138.0,930310138.0,93066.0,8,6,20,42,12,88,890.0,41.700756,88.229133,"POLYGON ((2.35813 48.9048, 2.35813 48.90474, 2..."
492,"{ ""lon"": 2.3601220243954284, ""lat"": 48.9390812...",None,971471,930302230.0,930302230.0,93066.0,127,5,0,0,0,132,297.0,57.607366,131.790633,"POLYGON ((2.36007 48.93914, 2.36011 48.93909, ..."
613,"{ ""lon"": 2.3716559323829625, ""lat"": 48.9418521...",None,971877,930299244.0,930299244.0,93066.0,47,4,3,7,30,91,718.0,44.288657,91.076052,"POLYGON ((2.37172 48.94183, 2.37172 48.9418, 2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075300,"{ ""lon"": 2.3755250187671768, ""lat"": 48.9408816...",None,889287,930301007.0,930301007.0,93066.0,2,1,0,0,0,3,690.0,8.664888,3.060450,"POLYGON ((2.37554 48.94088, 2.37551 48.94087, ..."
1075695,"{ ""lon"": 2.3761179299949657, ""lat"": 48.9396885...",None,900474,930301102.0,930301102.0,93066.0,8,6,22,29,4,69,869.0,33.067579,68.266061,"POLYGON ((2.37612 48.93965, 2.37607 48.93964, ..."
1075704,"{ ""lon"": 2.3496398245637113, ""lat"": 48.9358866...",None,887503,930303635.0,930303635.0,93066.0,34,14,20,6,1,75,724.0,51.245270,77.315867,"POLYGON ((2.34967 48.93579, 2.34962 48.93579, ..."
1075705,"{ ""lon"": 2.3793291458609791, ""lat"": 48.9346511...",None,887505,930305314.0,930305314.0,93066.0,13,14,28,39,16,110,864.0,54.153225,109.762669,"POLYGON ((2.37943 48.93461, 2.37946 48.9346, 2..."


## Exploration

In [9]:
toponymie_bati = gpd.read_file(GPKG, layer="toponymie_bati")
type(toponymie_bati)

geopandas.geodataframe.GeoDataFrame

In [10]:
toponymie_bati

,cleabs_de_l_objet,classe_de_l_objet,nature_de_l_objet,graphie_du_toponyme,source_du_toponyme,statut_du_toponyme,date_du_toponyme,langue_du_toponyme,geometry
0,CONSPONC0000000003752983,Construction ponctuelle,Calvaire,le bon dieu noir,IGN,Validé,2001-02-08,None,POINT (658587.8 6875503.7)
1,CONSPONC0000000003752947,Construction ponctuelle,Croix,croix de montmorency,IGN,Validé,2001-02-08,None,POINT (663943.9 6877958)
2,CONSPONC0000000004264735,Construction ponctuelle,Croix,croix saint-nicolas,IGN,Validé,1999-12-16,None,POINT (668379.2 6853628.1)
3,CONSPONC0000000068587303,Construction ponctuelle,Croix,croix rouge,IGN,Validé,2001-02-08,None,POINT (663512.9 6882417.7)
4,CONSPONC0000000316128944,Construction ponctuelle,Croix,croix de javot,Terrain,Collecté,1900-01-01,None,POINT (665320.2 6854692.2)
...,...,...,...,...,...,...,...,...,...
274,CIMETIER0000002344834285,Cimetière,Civil,cimetière musulman de bobigny,Mairie,Collecté,2024-04-03,None,POINT (658449.92 6866892)
275,CIMETIER0000000247645393,Cimetière,Civil,cimetière ancien,Mairie,Collecté,2024-04-03,None,POINT (667015.184 6868424.95)
276,CIMETIER0000000317228093,Cimetière,Civil,cimetière de l'orée du bois,Mairie,Collecté,2024-11-26,None,POINT (653940.277 6879224.45)
277,CIMETIER0000000317228093,Cimetière,Civil,cimetière antoine espiasse,Mairie,Collecté,2024-11-26,None,POINT (653940.277 6879224.45)


# CALCUL DE PROXIMITE

In [40]:
# Focus sur les métadonnées que l'on peut croiser celles de la BDD TOPO
annuaire[(annuaire.nom_etablissement=='Ecole élémentaire Daniel Sorano') & (annuaire.code_commune=='93066')][["nom_etablissement","type_etablissement"]]


,nom_etablissement,type_etablissement
41191,Ecole élémentaire Daniel Sorano,Ecole


In [42]:
zone_d_activite[(zone_d_activite.toponyme.str.contains("Daniel Sorano")==True)&(zone_d_activite.methode_d_acquisition_planimetrique=="BDTopo")][["toponyme","nature","nature_detaillee","date_modification"]]


,toponyme,nature,nature_detaillee,date_modification
9968,École Élémentaire Daniel Sorano,Enseignement primaire,Groupe scolaire,2024-04-17 14:57:25.903
